# MCMC3.0: Kitaev Model

## Introduction

I will give you the first physical application to the Kitaev model simulation based on
https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.113.197205.

In [1]:
using ResumableFunctions
using SparseArrays
using LinearAlgebra

First, these are the necessary libraries. If you have not installed them yet, type `]` in REPL and `add` these packages.

In [2]:
const Jx = 1 / 3 # oppposite sign to Motome's
const Jy = 1 / 3
const Jz = 1 / 3

0.3333333333333333

The sign convention follows Kitaev's original paper, not Motome's.

In [3]:
"""
Metropolis method

You can replace log(rand()) by log(1.0 - rand()) if you do not want to see Inf.
"""
function Metropolis(βF::Float64, βFnew::Float64)::Bool
    βF - βFnew > log(rand())
end

Metropolis

This is a simpler version of the `Metropolis` function used in MCMC1.0.

## Boundary condition

In order to use the Jordan-Wigner transformation, I employ an open boundary condition.

In [4]:
"""
Generating a honeycomb lattice with an open boundary condition.
"""
function openhoneycomb(Lx::Int64, Ly::Int64)::Tuple
    N = 2Lx * Ly
    # generating nearest neighbor bonds
    # the first part is odd, the second part is even, and they are zipped directly
    nnx = zip(1 : 2 : (N - 1), 2 : 2 : N)
    nny = Iterators.flatten((zip((1 + 2i) : 2Lx : (2Lx * (Ly - 1)  + 1 + 2i), 2i : 2Lx : (2Lx * (Ly - 1)  + 2i)) for i in 1 : (Lx - 1)))
    nnz = zip(1 : 2 : (N - 1), Iterators.flatten(((2Lx + 2) : 2 : N, 2 : 2 : 2Lx)))
    # generating a pair of z-bonds included in each hexagonal plaquette
    plaquette = Iterators.flatten(zip((Lx * (i - 1) + 1) : (Lx * (i - 1) + Lx - 1), (Lx * (i - 1) + 2) : (Lx * (i - 1) + Lx)) for i in 1 : Ly)
    N, nnx, nny, nnz, plaquette
end

openhoneycomb

I will illustrate how this code works to generate a honeycomb lattice.

~ under construction ~

## Main routine

In [5]:
@resumable function measurementflux(method::Function, lattice::Function, β::Float64,
        J₁::Float64, J₂::Float64, J₃::Float64, Lx::Int64, Ly::Int64)::Float64
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(J₁, nnx), (J₂, nny), (J₃, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J # if you do not know why I multiply 2 here, check Kitaev's original paper
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0 # any positive number is ok
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            ev = eigvals(Hermitian(hdense))
            positiveev = Iterators.drop(ev, N >> 1)
            βFnew = -sum((log(2.0 * cosh(β * ϵ / 2.0)) for ϵ in positiveev))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        @yield sum((η[k] * η[l] for (k, l) in plaq)) / Np
    end
end

measurementflux (generic function with 1 method)

Let's discard the first 10000 samples for thermalization.

In [6]:
mcstep = Iterators.drop(measurementflux(Metropolis, openhoneycomb, 1.0, Jx, Jy, Jz, 4, 4), 10000)

Base.Iterators.Drop{getfield(Main, Symbol("##362"))}(getfield(Main, Symbol("##362"))(0x00, #undef, 4, #undef, #undef, #undef, #undef, #undef, 0.3333333333333333, getfield(Main, Symbol("##8#11")){Float64}(2.164697126e-314), Metropolis, #undef, #undef, #undef, #undef, openhoneycomb, #undef, #undef, #undef, getfield(Main, Symbol("##7#10"))(), #undef, #undef, #undef, #undef, 4, 1.0, 0.3333333333333333, #undef, #undef, #undef, #undef, #undef, #undef, #undef, 0.3333333333333333, #undef, #undef, #undef, #undef), 10000)

I'll take 10000 samples from now on.

In [7]:
Nsample = 10000

10000

Here I directly get a mean value by summation. Next time I will use a smarter way.

In [8]:
sum(Iterators.take(mcstep, Nsample)) / Nsample

-0.0015999999999999936

## More on mesurements

In order to simplify statistical operations, use `Statistics`. In order to apply such statistical functions, it is better to transform the iterator to a vector before going on.

In [9]:
using Statistics
v = collect(Iterators.take(mcstep, Nsample))
v[1 : 10]

10-element Array{Float64,1}:
  0.3333333333333333 
  0.3333333333333333 
  0.0                
 -0.16666666666666666
  0.3333333333333333 
  0.16666666666666666
  0.0                
 -0.16666666666666666
 -0.5                
  0.0                

You can use `stdm` or `std` to calculate a "sample" standard deviation.

In [10]:
m = mean(v)
s = stdm(v, m) / sqrt(length(v))
println("$m ± $s")

-0.0012500000000000007 ± 0.0028837394339797907


At low temperature, the standard deviation does not decrease rapidly due to the strong autocorrelation. This can be solved by binning. Bins will be discussed in MCMC3.5.

If you specify `corrected = false`, it is a standard deviation divided by `n = length(v)`.

In [11]:
stdm(v, m, corrected = false) != s

true

Instead of saving memory, you can use `@.` macro to rewrite the code by arrays.

In [12]:
@resumable function measurementEf(method::Function, lattice::Function, β::Float64,
        J₁::Float64, J₂::Float64, J₃::Float64, Lx::Int64, Ly::Int64)::Vector{Float64}
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(J₁, nnx), (J₂, nny), (J₃, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 2.0im * J
        h[nn[2], nn[1]] = -2.0im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0 # any positive number is ok
    β₂ = β * 0.5
    hdense = Array(h)
    #plaq = collect(plaquette)
    #Np = length(plaq)
    ev = zeros(Float64, N)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            evnew = eigvals(Hermitian(hdense))
            βFnew = -sum(@. log(exp(β₂ * evnew[(N >> 1 + 1) : end]) + exp(-β₂ * evnew[(N >> 1 + 1) : end])))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
                ev .= evnew # be careful "." is necessary
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        Ef = -sum(@. ev[(N >> 1 + 1) : end] * tanh(β₂ * ev[(N >> 1 + 1) : end] )) * 0.5
        ∂Ef∂β = -sum(@. (ev[(N >> 1 + 1) : end] * sech(β₂ * ev[(N >> 1 + 1) : end])) ^ 2) * 0.25
        # you can also use automatic differentiation
        @yield [Ef, ∂Ef∂β] # Tuple does not work in Statistics.jl
    end
end

measurementEf (generic function with 1 method)

In [13]:
const β = 10.0

10.0

A new function yields `Ef` and `∂Ef∂β` at the same time. From this you can calculate the specific heat.

In [14]:
mcstep2 = Iterators.drop(measurementEf(Metropolis, openhoneycomb, β, Jx, Jy, Jz, 4, 4), 10000)

Base.Iterators.Drop{getfield(Main, Symbol("##364"))}(getfield(Main, Symbol("##364"))(0x00, #undef, 4, #undef, #undef, #undef, #undef, getfield(Main, Symbol("##19#20"))(), #undef, 0.3333333333333333, Metropolis, #undef, #undef, #undef, #undef, #undef, openhoneycomb, #undef, #undef, #undef, #undef, #undef, 3.66e-322, 4, 10.0, 0.3333333333333333, #undef, #undef, #undef, #undef, #undef, #undef, #undef, 0.3333333333333333, #undef, #undef, #undef, #undef), 10000)

The formula for the specific heat $C_v$ is

~ under construction ~

In [15]:
function TTCv(v::Vector{Float64})::Vector{Float64}　#T^2 Cv = meanTTCv[1] - meanTTCv[2]^2
    [v[1] ^ 2 - v[2], v[1]]
end

TTCv (generic function with 1 method)

In [16]:
meanTTCv = mean(TTCv, Iterators.take(mcstep2, Nsample))

2-element Array{Float64,1}:
 59.185653962748134
 -7.692223043546322

In [17]:
Cv = (β ^ 2) * (meanTTCv[1] - meanTTCv[2] ^ 2)

1.5358611083087226

For the values like the specific heat, it is not recommended to calculate the errorbar directly by using e.g. automatic differentiation. I recommend the jackknife method instead. However, for simple expectation values like the internal energy, you can easily calculate its standard deviation by Statistics.jl.

In [18]:
collectEf = collect(Iterators.take(mcstep2, Nsample))
meanEf = mean(collectEf)
stdEf = stdm(collectEf, meanEf) / sqrt(length(collectEf))
println("Ef = $(meanEf[1]) ± $(stdEf[1]), ∂Ef∂β = $(meanEf[2]) ± $(stdEf[2])")

Ef = -7.69128286625565 ± 0.0006669091710654152, ∂Ef∂β = -0.010776849271435898 ± 2.5969182726986748e-5


**Exercise**: implement a Type-II boundary condition of the honeycomb lattice in the original paper.